## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SBBSSBSB' # my file
#file_index='BBBSBBBB'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1832
+-------------------+------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+------------------+-----------+--------------------+------+--------------------+------+
|            coeff_1|           coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|          res_mean|    station|           total_var|undefs|              vector|  year|
+-------------------+------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+------------------+-----------+--------------------+------+--------------------+------+
|-2507.0190628882583|-983.4649329584822| -512.2153899884555|   1314.6|SBBSSBSB| 43.6483|-108.2036|       TOBS|0.30388174153029657| 0.1967580548461491|0

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1, coeff_2, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1, coeff_2, coeff_3 FROM weather


station    year      coeff_1      coeff_2     coeff_3
0  USC00488875  1924.0 -2507.019063  -983.464933 -512.215390
1  USC00482725  1987.0 -2622.131182  -857.781976 -415.419322
2  USC00487260  1963.0 -1810.084582  -803.082779 -522.556106
3  USC00488880  1984.0 -2354.902225 -1172.017187 -347.434860
4  USC00486195  1973.0 -2779.946096  -667.127579 -480.669830

In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table.head(10)

station  USC00480140  USC00480237  USC00480324  USC00480528  USC00480603  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0  -1390.630702          NaN          NaN          NaN          NaN   

station  USC00480605  USC00480778  USC00480865  USC00481000  USC00481284  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USS0009F28S  USS0009G03S  USS0010F09S  USS0010F15S  \
year        ...                                                            
1901.0      ...               NaN          NaN          NaN          NaN   
1902.0      ...               NaN          NaN          NaN          NaN   
1903.0      ...               NaN          NaN          NaN          NaN   
1904.0      ...               NaN          NaN          NaN          NaN   
1905.0      ...               NaN          NaN          NaN          NaN   
1906.0      ...               NaN          NaN          NaN          NaN   
1907.0      ...               NaN          NaN          NaN          NaN   
1908.0      ...               NaN          NaN          NaN          NaN   
1909.0      ...               NaN          NaN          NaN          NaN   
1910.0      ...               NaN          NaN          NaN          NaN   

station  USS0010F16S  USS0010F17S  USS0010F19S  USS0010F23S  USS0010F29S  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN          NaN          NaN   

station  USS0010G02S  
year                  
1901.0           NaN  
1902.0           NaN  
1903.0           NaN  
1904.0           NaN  
1905.0           NaN  
1906.0           NaN  
1907.0           NaN  
1908.0           NaN  
1909.0           NaN  
1910.0           NaN  

[10 rows x 75 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time versus location on the first eigenvector coefficient for SNWD, we compute the Root Mean Square (RMS) before and after subtracting the mean-by-station and the mean-by-year:

<pre>
total RMS                          =  4624.98
RMS after removing mean-by-station =  1745.65
RMS after removing mean-by-year    =  2682.17
</pre>

Subtracting the mean-by-station has the effect of removing the effect of station on RMS, so that the RMS remaining is due entirely to different years.  Similarly, the RMS remaining after removing mean-by-year accounts for the effect of different stations.  Since we have a higher RMS after removing mean-by-year, we conclude that station has a larger impact on the first eigenvector coefficient than year.  This makes intuitive sense, as we would expect certain stations to consistently have more snow depth than other stations, regardless of year.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))


for i in range(1,4):
    meas='coeff_'+str(i)
    year_station_table=pdf.pivot(index='year', columns='station', values=meas)

    mean_by_year=np.nanmean(year_station_table,axis=1)
    mean_by_station=np.nanmean(year_station_table,axis=0)
    #print mean_by_station.shape
    #print mean_by_year.shape
    tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
    tbl_minus_station = year_station_table-mean_by_station

    RMS_tot = RMS(year_station_table)
    RMS_st = RMS(tbl_minus_station)
    RMS_yr = RMS(tbl_minus_year)

    print 'Coefficient %s:' % str(i)
    print 'total RMS                          = %.2f' % RMS_tot
    print 'RMS after removing mean-by-station = %.2f, fraction explained = %.1f' % (RMS_st, 100.0*(1.0-RMS_st/RMS_tot)) # what remains is variation across years
    print 'RMS after removing mean-by-year    = %.2f, fraction explained = %.1f\n' % (RMS_yr, 100.0*(1.0-RMS_yr/RMS_tot)) # what remains is variation across stations

Coefficient 1:
total RMS                          = 1651.88
RMS after removing mean-by-station = 434.16, fraction explained = 73.7
RMS after removing mean-by-year    = 730.79, fraction explained = 55.8

Coefficient 2:
total RMS                          = 872.00
RMS after removing mean-by-station = 152.98, fraction explained = 82.5
RMS after removing mean-by-year    = 150.82, fraction explained = 82.7

Coefficient 3:
total RMS                          = 403.71
RMS after removing mean-by-station = 154.24, fraction explained = 61.8
RMS after removing mean-by-year    = 116.65, fraction explained = 71.1



### For report


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(10):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 403.709721296
0 after removing mean by year    = 116.650346175
0 after removing mean by stations= 94.15733909
1 after removing mean by year    = 93.395434282
1 after removing mean by stations= 93.2742401334
2 after removing mean by year    = 93.2359312542
2 after removing mean by stations= 93.2216739905
3 after removing mean by year    = 93.2160387842
3 after removing mean by stations= 93.2137105782
4 after removing mean by year    = 93.2127095548
4 after removing mean by stations= 93.2122635599
5 after removing mean by year    = 93.2120587068
5 after removing mean by stations= 93.2119622331
6 after removing mean by year    = 93.2119158874
6 after removing mean by stations= 93.2118932749
7 after removing mean by year    = 93.2118821083
7 after removing mean by stations= 93.211876542
8 after removing mean by year    = 93.2118737466
8 after removing mean by stations= 93.2118723342
9 after removing mean by year    = 93.2118716169
9 after removing mean by stations= 93.21187125